In [58]:
from dotenv import load_dotenv
import os
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [59]:
import pandas as pd
data = pd.read_csv('amazon.csv')

In [60]:
data.isnull().sum()

product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
rating                 0
rating_count           2
about_product          0
user_id                0
user_name              0
review_id              0
review_title           0
review_content         0
img_link               0
product_link           0
dtype: int64

In [61]:
data['rating_count'].fillna("0", inplace=True)

C:\Users\JAIVAL CHAUHAN\AppData\Local\Temp\ipykernel_23452\3612801901.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['rating_count'].fillna("0", inplace=True)


In [62]:
from pinecone import Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY, environment="us-east-1")
index = pc.Index("auracart-product-search")

In [63]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

In [64]:
vectors = []
for i, row in data.iterrows():
    text = row['product_name'] + " " + row['about_product']
    vector = model.encode(text).tolist()
    vectors.append((str(row['product_id']), vector, row.to_dict()))

In [65]:
for i in range(0, len(vectors), 100):  # Batch size 100
    index.upsert(vectors=vectors[i:i+100])

In [66]:
query = "tv"
query_vector = model.encode(query).tolist()
results = index.query(vector=query_vector, top_k=5, include_metadata=True)
print(results['matches'])

[{'id': 'B08XXF5V6G',
 'metadata': {'about_product': 'Resolution : Ultra HD 4k (3840 X 2160) '
                               'resolution | Refresh Rate : 60 Hertz | 178 '
                               'Degree wide viewing angle|Connectivity: 3 HDMI '
                               'ports to connect set top box, Blu-ray speakers '
                               'or a gaming console | 2 USB ports to connect '
                               'hard drives or other USB devices|Sound: 30 '
                               'Watts Output | Powerful Speakers with Digital '
                               'Dolby Plus Audio | DTS Tru Surround for '
                               'remarkable sound quality|Smart TV Features : '
                               'Kodak LED TV provides experience that brings '
                               'together your favorite streaming content on '
                               'the home screen. Choose from a vast catalog of '
                               'movies 